<div align="center">

# Применение мамшинного обучения для смыслового анализа текста
## Подготовка набора данных с обзорами фильмов на IMDB

</div>

---

### Обработка естественного языка (Natural Language Processing, NLP)
**NLP** - так называемый анализ эмоциональной окраски текстов (анализ мнений или анализ настроений), с помощью которого мы можем максимально эффективно извлекать полезные данные для анализа.

**Цель** - взять 50.000 отзывов на IMDB, извлечь значимую информацию из подмножества обзоров фильмов и построить модель ML, способную предсказать, понравится или не понравится фильм определенному пользователю.

In [86]:
import pyprind
import pandas as pd
import os
import sys
import time
import re
from tqdm import tqdm 
import numpy as np
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
# Соберем отдельные текстовые документы в один файл CSV

basepath = os.path.expanduser('~/ML/Data/aclImdb')
labels = {"pos": 1, "neg": 0}

data = []

# Считаем общее количество файлов
total_files = sum(len(os.listdir(os.path.join(basepath, s, l))) 
                  for s in ('train', 'test') 
                  for l in ('pos', 'neg'))

with tqdm(total=total_files, desc="Загрузка отзывов") as pbar:
    for s in ('train', 'test'):
        for l in ('pos', 'neg'):
            folder = os.path.join(basepath, s, l)
            for filename in os.listdir(folder):
                with open(os.path.join(folder, filename), 'r', encoding='utf-8') as f:
                    review = f.read()
                data.append([review, labels[l]])
                pbar.update(1)


df = pd.DataFrame(data, columns=['review', 'sentiment'])

Загрузка отзывов: 100%|████████████████| 50000/50000 [00:00<00:00, 59278.51it/s]


Используя вложенные циклы, мы прошлись по подкаталогам `train` и `test`, в главном каталоге `aclImdb` и прочитали отдельные текстовые файлы из подкаталогов `pos` и `neg`, которые добавили к набору данных `df` вместе с целочисленной меткой класса (1 = положительный и 0 = отрицательный).

Так как метки классов в собраном наборе данных упорядочены, надо их перемешать, что полезно для разделения набора данных на `train` и `test`.

In [6]:
print(df.head())
print(df.shape)

                                              review  sentiment
0  This movie is goofy as hell! I think it was wr...          1
1  Kurosawa, fresh into color, losses sight of hi...          1
2  A very addictive series.I had not seen an exac...          1
3  An excellent cast makes this movie work; all o...          1
4  This is one of the most brilliant movies that ...          1
(50000, 2)


In [7]:
# Сохраним собранный и перетасованный набор данных в CSV
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('~/ML/Data/movie_data.csv', index = False, encoding = 'utf-8')

In [9]:
# Удостоверимся, что сохранили данные в нужном формате
df = pd.read_csv('~/ML/Data/movie_data.csv', encoding = 'utf-8')
df.head(3)

,review,sentiment
0,"I just want to add my two cents worth, and for...",1
1,This video nasty was initially banned in Brita...,0
2,The original GRUDGE (the original American rem...,0


In [10]:
# Удостоверимся, что DataFrame содержит все строки
df.shape

(50000, 2)

## Знакомство с моделью мешка слов

### Модель "мешок слов" (Bag-of-Words, BoW)

#### Преобразование категориальных данных

* Перед передачей алгоритму машинного обучения **категориальные данные** (например, текст или слова) нужно **преобразовать в числовую форму**.


#### Идея модели мешка слов

* **Цель:** представить текст в виде **вектора числовых признаков**.
* **Основная идея:** не учитывать порядок слов, а только их наличие и частоту.

##### Этапы:

1. **Создание словаря уникальных токенов**

   * Из всего набора документов формируется словарь уникальных слов.

2. **Построение вектора признаков для каждого документа**

   * Вектор содержит **подсчет количества каждого слова** из словаря в данном документе.
   * Так как каждый документ использует только часть словаря, **вектор преимущественно состоит из нулей** → такие векторы называются **разреженными**.


#### Термины

* **Raw Term Frequencies (t.f{t, d}):**

  * Количество раз, когда термин $t$ встречается в документе $d$.
* **Особенность модели:**

  * Порядок слов в документе **не имеет значения**.
  * Порядок значений в векторе определяется **индексами словаря** (обычно в алфавитном порядке).


#### Инструмент для реализации

* **CountVectorizer** (например, в Python/Scikit-learn) позволяет автоматически:

  * Создать словарь;
  * Преобразовать документы в разреженные векторы частот слов.


In [15]:
# Пример

# Строим словарь уникальных токенов из всего набора даныых
count = CountVectorizer()
docs = np.array(['Солнце светит',
                 'Погода прекрасная',
                 'Солнце светит, погода прекрасная, а один плюс один будет два'])

# Построим словарь мешка слов и преобразуем в разряженные векторы признаков
bag = count.fit_transform(docs)

print(count.vocabulary_)

{'солнце': 7, 'светит': 6, 'погода': 4, 'прекрасная': 5, 'один': 2, 'плюс': 3, 'будет': 0, 'два': 1}


CountVectorizer сканирует все документы (docs) и находит уникальные слова (токены), в нашем случае уникальные слова.

>Индексы могут выглядеть произвольно (здесь CountVectorizer сам распределил их), но главное: каждое слово имеет уникальный индекс.

In [14]:
# Векторы признаков
print(bag.toarray())

[[0 0 0 0 0 0 1 1]
 [0 0 0 0 1 1 0 0]
 [1 1 2 1 1 1 1 1]]


### 2. Построение разреженных векторов признаков

`bag.toarray()` показывает **вектор частот слов для каждого документа**.

#### Документ 1: `'Солнце светит'`

* Слова: `['солнце', 'светит']`
* Вектор `[0 0 0 0 0 0 1 1]`

  * Индекс 6 → `'светит'` встречается 1 раз
  * Индекс 7 → `'солнце'` встречается 1 раз
  * Остальные слова не встречаются → 0

#### Документ 2: `'Погода прекрасная'`

* Вектор `[0 0 0 0 1 1 0 0]`

  * Индекс 4 → `'погода'` = 1
  * Индекс 5 → `'прекрасная'` = 1
  * Остальные слова = 0

#### Документ 3: `'Солнце светит, погода прекрасная, а один плюс один будет два'`

* Вектор `[1 1 2 1 1 1 1 1]`

  * `'будет'` (индекс 0) = 1
  * `'два'` (индекс 1) = 1
  * `'один'` (индекс 2) = 2 (слово встречается дважды)
  * `'плюс'` (индекс 3) = 1
  * `'погода'` (индекс 4) = 1
  * `'прекрасная'` (индекс 5) = 1
  * `'светит'` (индекс 6) = 1
  * `'солнце'` (индекс 7) = 1


### Вывод

* **Каждый документ** представлен **вектором длиной = числу уникальных слов во всём корпусе**.
* **Значения вектора = количество раз, которое слово встречается в документе** (raw term frequency).
* **Порядок слов в тексте не важен** — только частота.



### N-граммы

* **1-грамма (униграмма):** каждое слово отдельно.

  * Пример: `"the sun is shining"` → `"the"`, `"sun"`, `"is"`, `"shining"`
* **2-грамма:** пары слов.

  * Пример: `"the sun"`, `"sun is"`, `"is shining"`
* **n-грамма:** последовательность из n элементов.
* В Python (`CountVectorizer`) можно задать n через `ngram_range`.

---

<div align="center">

## Оценка релевантности слов с помощью частоты термина и обратной связи

</div>

---

Бывает так, что мы можем столкнуться со словами, которые встречаются в документах обоих классов, и они могут не содержать полезной информации, именно для этого случая существует метод **Обратная частота документа (TF-IDF)**


### TF-IDF (Term Frequency-Inverse Document Frequency)

* **Идея:** уменьшить вес часто встречающихся слов, которые мало помогают различать документы.
* **Формула:**

$$
\text{tf-idf}(t, d) = \text{tf}(t, d) \times \text{idf}(t, d)
$$

* **tf(t, d)** — частота слова $t$ в документе $d$
* **idf(t, d)** — обратная частота документа:

$$
\text{idf}(t, d) = \log \frac{n_d}{1 + df(d, t)}
$$

* $n_d$ — общее число документов в датасете

* $df(d, t)$ — число документов, где встречается слово $t$

* +1 в знаменателе и логарифм предотвращают слишком большой вес редких слов.

* В Python: `TfidfTransformer` берёт частоты из `CountVectorizer` и преобразует их в TF-IDF.

In [22]:
# Преобразование в TF-IDF необработанных частот терминов
tfidf = TfidfTransformer(use_idf = True, # Назначить нулевой вес
                         norm = 'l2', # L2-нормализация
                         smooth_idf = True)
np.set_printoptions(precision = 2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.   0.   0.   0.   0.   0.71 0.71]
 [0.   0.   0.   0.   0.71 0.71 0.   0.  ]
 [0.33 0.33 0.66 0.33 0.25 0.25 0.25 0.25]]


* **Нормализация TF-IDF** нужна, чтобы сравнивать документы разной длины.
* **TfidfTransformer** делает это автоматически (по умолчанию **L2-норма**).
* **L2-норма:** делит вектор признаков на его длину, чтобы итоговый вектор имел длину 1:

$$
\mathbf{v}_{\text{norm}} = \frac{\mathbf{v}}{\|\mathbf{v}\|_2} = \frac{\mathbf{v}}{\sqrt{v_1^2 + v_2^2 + \dots + v_n^2}}
$$

* Это позволяет сравнивать документы по **направлению векторного пространства**, а не по абсолютной длине текста.


### Зачем нужна нормализация TF-IDF

1. **Проблема:**

   * У нас есть два документа: один короткий, другой длинный.
   * Даже если в них встречаются одинаковые слова с одинаковой частотой, длинный документ даст **большие числа TF-IDF**, просто потому что слов больше.
   * Без нормализации длинный документ будет "весить" больше, хотя по сути информация та же.

2. **Решение (нормализация L2):**

   * Делим все числа TF-IDF в векторе на его **длину** (квадратный корень суммы квадратов всех элементов).
   * Вектор становится длины 1, но **сохраняется пропорция между словами**.

3. **Эффект:**

   * Теперь сравнивать документы можно по **направлению векторного пространства** (какие слова важнее), а не по количеству слов.
   * Длина документа больше не влияет на "вес" документа.

---

## Очистка текстовых данных

**Задача** - удалить знаки препинания и другие небуквенные символы, так как они не содержат полезной семантики, хоть и иногда запятые предоставляют полезную информацию. Оставим только символы смайликов, поскольку полезны для анализа эмоциональной окраски

In [28]:
# До очистки данных
df.loc[0, 'review'][-50:]

'olitics fairly funny, too!) Damon Runyon lives!!!!'

In [30]:
# Библиотека регулярных выражений для очистки

def preprocessor(text):
    # Убираем HTML-теги
    text = re.sub(r'<[^>]*>', '', text)
    
    # Находим смайлы
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Убираем всё, кроме букв и цифр, добавляем смайлы
    text = re.sub(r'[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    
    return text


### Предобработка текста с помощью регулярных выражений

1. **Удаление HTML-разметки**

   * Используем простое регулярное выражение `<.*?>` для очистки текста от тегов.
   * Для более сложного HTML можно использовать модуль `html.parser`.

2. **Обработка смайликов**

   * Сначала ищем смайлики с помощью регулярных выражений и временно сохраняем их.
   * После очистки текста от всех несловесных символов (`[\W]+`) добавляем смайлики в конец строки.
   * Символ «носа» в смайликах (`:-)`) удаляется для единообразия.

3. **Приведение текста к нижнему регистру**

   * Заглавные буквы не считаются информативными для анализа тональности (теряем информацию об именах собственных, но упрощаем обработку).


**Важно:** добавление смайликов в конец строки не нарушает модель мешка слов, так как порядок слов не учитывается.

In [31]:
preprocessor(df.loc[0, 'review'][-50:])

'olitics fairly funny too damon runyon lives '

In [32]:
# Применим нашу функцию ко всем документам в нашем DataFrame
df['review'] = df['review'].apply(preprocessor)

In [35]:
# Убедимся в этом
df.head(5)

,review,sentiment
0,i just want to add my two cents worth and forg...,1
1,this video nasty was initially banned in brita...,0
2,the original grudge the original american rema...,0
3,based on the manga comic of well known artist ...,1
4,i d been interested in watching this ever sinc...,0


<div align="center">

## Получение токенов из документов 

</div>

---


## Токенизация и стемминг

### Токенизация

* **Цель:** разбить текст на отдельные элементы (токены).
* **Простой способ:** разделить очищенный текст по пробелам (каждое слово = токен).

### Стемминг (stemming)

* **Что это:** преобразование слова в его корень (основу), чтобы объединять родственные слова.
* **Пример:** `running → run`, `cats → cat`.
* **Алгоритмы стемминга:**

  * **Porter (классический, старый, простой)**
  * **Snowball / Porter2 (быстрее, современнее)**
  * **Lancaster (агрессивный, короткие и иногда неясные корни)**
* Реализация в Python: пакет **NLTK** (`nltk.stem`)

### Лемматизация vs Стемминг

* **Лемматизация:** получает каноническую форму слова (грамматически правильную).
* **Стемминг:** быстрее, но может давать «несуществующие» слова.
* **На практике:** переход от стемминга к лемматизации почти не улучшает качество классификации текста.

### Удаление стоп-слов

* **Что это:** удаление слов, которые часто встречаются и мало помогают различать документы.
* **Примеры стоп-слов:** `is, has, and, like`
* **Когда полезно:** при работе с необработанными или нормализованными частотами терминов (не обязательно при TF-IDF, где частые слова уже получают меньший вес)
* **В Python:** NLTK содержит готовый набор стоп-слов: `nltk.download`.


In [38]:
# Токенизация

def tokenizer(text):
    return text.split()
    
#Пример
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [46]:
# Стемминг

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# Пример 
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [56]:
# Удаление стоп слова


stop = stopwords.words('english')
# Пример
[w for w in tokenizer_porter('a runner likes running and runs a lot')
  if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

<div align="center">

## Обучение модели логистической регрессии для классификации документов

</div>

---


## Классификация обзоров фильмов с помощью логистической регрессии

### Данные

* 25k документов → обучение
* 25k документов → тест

### Модель

* Логистическая регрессия (две категории: положительные / отрицательные).
* Решатель: **`liblinear`** (лучше подходит для больших наборов данных, чем `lbfgs`).

### Поиск параметров

* Используем **GridSearchCV** с 5-кратной стратифицированной кросс-валидацией.
* Ограничиваем число комбинаций (иначе обучение слишком долго).

### Векторизация текста

* Вместо `CountVectorizer + TfidfTransformer` → используем **`TfidfVectorizer`** (объединяет оба шага).

### Grid параметров

* **Вариант 1:** `use_idf=True`, `smooth_idf=True`, `norm='l2'` (TF-IDF).
* **Вариант 2:** `use_idf=False`, `smooth_idf=False`, `norm=None` (сырые частоты).

### Регуляризация

* Классификатор: регуляризация **L2**.
* Подбираем параметр **C** (обратная сила регуляризации).

In [64]:
# Разделяем данные на обучающую и тестовую выборки
X_train = df.loc[:25000, 'review'].values 
X_test = df.loc[25000:, 'review'].values   
y_train = df.loc[:25000, 'sentiment'].values  
y_test = df.loc[25000:, 'sentiment'].values   

# Определяем небольшой сет параметров для GridSearchCV
small_param_grid = [
    {
        # Векторизация только униграмм (по одному слову)
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],  # не удаляем стоп-слова
        'vect__tokenizer': [tokenizer, tokenizer_porter],  # используем два разных токенизатора
        'clf__penalty': ['l2', 'l1'],  # разные виды регуляризации
        'vect__use_idf': [True],       # использовать TF-IDF
        'vect__smooth_idf': [True],    # сглаживание IDF
        'vect__norm': ['l2'],          # L2-нормализация
        'clf__C': [1.0, 10.0]          # разные коэффициенты регуляризации
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],  # пробуем удалять стоп-слова или нет
        'vect__tokenizer': [tokenizer],    # только один токенизатор
        'vect__use_idf': [False],          # не использовать TF-IDF
        'vect__smooth_idf': [False],       # без сглаживания
        'vect__norm': [None],              # без нормализации
        'clf__penalty': ['l2', 'l1'],      # регуляризация
        'clf__C': [1.0, 10.0]              # коэффициенты регуляризации
    },
]

# Создаем пайплайн: сначала TF-IDF преобразование, затем логистическая регрессия
lr_tfidf = Pipeline([
    ('vect', TfidfVectorizer(strip_accents=None,   # не удаляем акценты
                             lowercase=False,     # не приводим к нижнему регистру
                             preprocessor=None,   # не используем дополнительный препроцессор
                             token_pattern=None)),# токенизация через кастомные токенизаторы
    ('clf', LogisticRegression(solver='liblinear'))  # логистическая регрессия
])

start = time.time()

# Настраиваем GridSearchCV для подбора лучших гиперпараметров
gs_lr_tfidf = GridSearchCV(
    lr_tfidf,               # модель
    small_param_grid,       # сет параметров
    scoring='accuracy',     # метрика для оценки качества
    cv=5,                   # кросс-валидация с 5 фолдами
    verbose=1,              # минимальный вывод прогресса
    n_jobs=-1               # использовать все доступные ядра процессора
)

gs_lr_tfidf.fit(X_train, y_train)
end = time.time()

print(f"Время обучения: {(end - start)/60:.2f} минут ({end - start:.2f} секунд)")

print(f"Лучшие параметры: {gs_lr_tfidf.best_params_}")

print(f"Лучшая точность на кросс-валидации: {gs_lr_tfidf.best_score_:.4f}")


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Время обучения: 11.62 минут (697.29 секунд)
Лучшие параметры: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__norm': 'l2', 'vect__smooth_idf': True, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x750a509793a0>, 'vect__use_idf': True}
Лучшая точность на кросс-валидации: 0.8949


In [66]:
# Тест оценка на лучшей модели
clf = gs_lr_tfidf.best_estimator_
print(f'Точность на тествовом наборе: {clf.score(X_test, y_test):.3f}')

Точность на тествовом наборе: 0.901


Наилучшие результаты показала модель логистической регрессии с обычным `tokenizer` (без стемминга Портера и без стоп-слов), которая использовала **TF-IDF** для представления текста. Регуляризация была **L2** с параметром `C = 10.0`.

* Точность на обучении: 89,5%
* Точность на тесте: 90%

То есть модель способна с высокой точностью предсказывать, является отзыв положительным или отрицательным.

Наивный байесовский классификатор также широко используется для задач классификации текста, например, для фильтрации спама. Он прост в реализации, эффективен и хорошо работает на небольших наборах данных.

<div align="center">

## Работа с большими данными: онлайн-алгоритмы и внешнее обучение

</div>

---


### Проблема

* Полное построение векторов признаков для 50 000+ документов требует много памяти.
* Реальные наборы данных могут быть ещё больше и не помещаться в RAM.

### Решение: внешнее обучение (out-of-core learning)

* Обучение на данных, которые не умещаются в память, по частям (мини-пакетами).
* Используется **`partial_fit`** у `SGDClassifier` в scikit-learn.

### Основные шаги

1. **Tokenizer** – очищает текст, разбивает на токены, удаляет стоп-слова.
2. **Stream\_docs** – генератор, возвращающий документы по одному.
3. **Get\_minibatch** – получает из потока определённое количество документов.

### Ограничения обычных векторизаторов

* `CountVectorizer` и `TfidfVectorizer` требуют хранения всего словаря и всех векторов признаков в памяти → не подходят для внешнего обучения.

### Решение для внешнего обучения

* Используется **`HashingVectorizer`**:

  * Не хранит словарь в памяти
  * Использует хеш-функцию (например, `murmurHash3`) для генерации индексов признаков

In [68]:
stop = stopwords.words('english')

def tokenized(text):
    # Удаляем HTML-теги
    text = re.sub(r'<[^>]*>', '', text)

    # Находим смайлики
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)

    # Удаляем все несловесные символы и приводим к нижнему регистру
    text = re.sub(r'[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', '')

    # Разбиваем на токены и убираем стоп-слова
    tokenized = [w for w in text.split() if w not in stop]

    return tokenized


In [73]:
# Генератор возвращающий документы по одному
def stream_docs(path):
    path = os.path.expanduser(path)
    with open(path, 'r', encoding = 'utf-8') as csv:
        next(csv) # пропуск заголовка
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

# Проверим эту функцию прочитав первый документ из наших данных
# который должен вернуть кортеж из текста и метки класса
next(stream_docs(path = '~/ML/Data/movie_data.csv'))

('"I just want to add my two cents worth, and forgive me if I am repeating something that has already been posted, but I feel it is worth reminding people of the everlovin\' genius of Damon Runyon. Without the wonderfully street, hilarious writings of Damon Runyon this film would never have been made - nor most of the other great classics that deal with gamblers & the like from before 1960. Damon Runyon worked as a newspaper man, and he was from Colorado, but he sure did _get_ the street scene of the East Coast. If you are not a dedicated fan of old Hollywood comedies, I would recommend a few other flicks from Damon Runyon\'s writings; ""the Lemondrop Kid,"" and ""Little Miss Marker,"" both feature Bob Hope, who, aside from his politics, has always been a funny man. (As a West Coast liberal, I find his politics fairly funny, too!) Damon Runyon lives!!!!"',
 1)

In [74]:
# Определим функцию которая будет получать поток документов из
# функции генератора возвращающих документов и возвращать
# нужное кол-во документов через size
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [82]:
# Внешнее обучения с HashingVectorizer
vect = HashingVectorizer(decode_error = 'ignore',
                        n_features = 2**21, # Кол-во признаков
                        preprocessor = None,
                        tokenizer = tokenized)
# Стохастический градиентный спуск с параметром потерь
# логистической регрессии
clf = SGDClassifier(loss = 'log_loss', random_state = 1)
doc_stream = stream_docs(path = '~/ML/Data/movie_data.csv')

>Выбирая большое кол-во признаков в `HashingVectorizer`, мы уменьньшаем вероятность возникновения коллизий хешей, но при этом увеличиваем кол-во коэффицентов в нашей модели логистической регрессии.

In [83]:
# После настройки всех функций можем обучаться на внешних данных
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size = 1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes = classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16


In [85]:
# Оценка точности модели
X_test, y_test = get_minibatch(doc_stream, size = 5000)
X_test = vect.transform(X_test)
print(f'Точность: {clf.score(X_test, y_test):.3f}')

Точность: 0.877


### Онлайн-обучение модели (mini-batch)

* Используем **PyPrind** для отображения прогресса обучения (45 делений).
* Каждый мини-пакет = 1000 документов.
* Последние 5000 документов используются для оценки точности.
* **Ошибка NoneType:** возникает, если повторно вызвать генератор мини-пакетов — документы уже извлечены, поэтому `x_test` возвращает `None`. Решение: заново инициализировать генератор через `stream_docs(...)`.
* **Точность модели:** \~87% (немного ниже, чем при использовании GridSearchCV).
* Преимущество: эффективное использование памяти, обучение занимает <1 минуты.

### Алгоритм Word2Vec

* **Тип:** обучение без учителя, нейронная сеть.
* **Идея:** слова с похожим смыслом помещаются в близкие векторные кластеры.
* **Особенность:** позволяет выполнять логические операции в векторном пространстве, например:

  ```
  король - мужчина + женщина ≈ королева
  ```


<div align="center">

## Моделирование тем с использованием скрытого распределения Дирихле

</div>

---


## Моделирование тем (Topic Modeling)

* **Цель:** присвоить немаркированным документам темы или категории (например, спорт, финансы, политика).
* **Тип задачи:** обучение без учителя (классификация/кластеризация документов).

## Latent Dirichlet Allocation (LDA)

* **Что это:** генеративная вероятностная модель для выявления групп слов, часто встречающихся вместе — эти группы отражают темы.
* **Примечание:** не путать с Linear Discriminant Analysis (LDA) — метод контролируемого уменьшения размерности.
* **Входные данные:** матрица мешка слов (Bag-of-Words).
* **Выход:** две матрицы:

  1. **Документ-тема** — насколько каждый документ относится к каждой теме.
  2. **Слово-тема** — насколько каждое слово связано с каждой темой.
* **Идея:** перемножением двух матриц восстанавливается исходная матрица мешка слов с минимальной ошибкой.
* **Особенность:** нужно заранее задать количество тем (гиперпараметр).

## Реализация LDA в scikit-learn

* Класс: `LatentDirichletAllocation`
* Пример применения: анализ и классификация набора обзоров фильмов.
* В примере: ограничение на **10 тем**.

In [87]:
# Загрузка данных
df = pd.read_csv('~/ML/Data/movie_data.csv', encoding = 'utf-8')

# Применение CountVectorizer для создания матрицы набора слов
# которая потсупит на вход LDA
count = CountVectorizer(stop_words = 'english',
                        max_df = .1,
                        max_features = 5000)
X = count.fit_transform(df['review'].values)

# Обучение оценщика на матрице набора слов
lda = LatentDirichletAllocation(n_components = 10,
                                random_state = 123,
                                learning_method = 'batch')
X_topics = lda.fit_transform(X)

In [88]:
# Матрица, содержащая важность слова
lda.components_.shape

(10, 5000)

### Настройка LDA и фильтрация слов

### Ограничение частоты слов

* `max_df=0.1` — исключаем слова, которые встречаются в более чем 10% документов.

  * **Причина:** такие слова слишком общие и с меньшей вероятностью отражают конкретную тему.
* `max_features=5000` — учитываем только 5000 наиболее часто встречающихся слов.

  * **Причина:** ограничение размерности улучшает вычислительную эффективность и качество вывода модели.
* **Примечание:** значения `max_df` и `max_features` выбраны произвольно и могут быть настроены при сравнении результатов.

### Параметры обучения LDA

* `learning_method='batch'` — модель обучается на всех данных за одну итерацию.

  * Медленнее, чем `online` (мини-пакеты), но обычно точнее.
* `learning_method='online'` — онлайн-обучение (мини-пакеты).

### Алгоритм обучения

* Используется **Expectation-Maximization (EM)** для итеративного обновления оценок параметров.

### Результаты обучения

* После обучения можно получить матрицу `components_` экземпляра `lda`:

  * **Размерность:** количество слов × количество тем (например, 5000 × 10).
  * **Содержание:** важность каждого слова для каждой темы (в порядке возрастания по темам).

* Пример: обучение на матрице мешка слов с 10 темами может занять до 5 минут на ноутбуке.

In [90]:
# Выведем пять самых важных слов для каждой из 10 тем
# Значения важности слов ранжируются в порядке возрастания
# Поэтому отсортируем массив тем в обратном порядке
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Тема {(topic_idx + 1)}:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                    [:-n_top_words -1:-1]]))

Тема 1:
horror effects budget special gore
Тема 2:
guy money girl stupid girls
Тема 3:
version english action japanese match
Тема 4:
book read documentary game music
Тема 5:
series tv episode shows family
Тема 6:
family woman father mother wife
Тема 7:
music role performance musical star
Тема 8:
war police murder action men
Тема 9:
script actor worst minutes awful
Тема 10:
comedy action watched original fun


Краткое описание каждой темы на основе ключевых слов `LDA`:

| №  | Название темы          | Ключевые слова                            | Краткое описание                                                           |
| -- | ---------------------- | ----------------------------------------- | -------------------------------------------------------------------------- |
| 1  | Ужасы                  | horror, effects, budget, special, gore    | Ужасы с акцентом на спецэффекты, кровь и бюджетные аспекты                 |
| 2  | Молодежь и отношения   | guy, money, girl, stupid, girls           | Истории о парнях и девушках, социальные и повседневные ситуации            |
| 3  | Боевики и экшн         | version, english, action, japanese, match | Боевики, версии фильмов на английском или японском, спортивные элементы    |
| 4  | Документальные и книги | book, read, documentary, game, music      | Документальные фильмы, книги, музыка и игры                                |
| 5  | Телесериалы            | series, tv, episode, shows, family        | Телесериалы и шоу, эпизоды, семейные истории                               |
| 6  | Семья                  | family, woman, father, mother, wife       | Семейные темы, женские и отцовские роли, взаимоотношения в семье           |
| 7  | Музыка и мюзиклы       | music, role, performance, musical, star   | Музыкальные фильмы и мюзиклы, роли, выступления, знаменитости              |
| 8  | Криминал и боевики     | war, police, murder, action, men          | Боевики и криминальные сюжеты: война, полиция, убийства, мужские персонажи |
| 9  | Критика фильмов        | script, actor, worst, minutes, awful      | Критика фильмов: сценарий, актеры, хронометраж, негативные аспекты         |
| 10 | Комедии                | comedy, action, watched, original, fun    | Комедийные фильмы, боевики с юмором, оригинальные и развлекательные        |


Чтобы убедиться, что категории осмысленно связаны с реальными обзорами, выведем текст отзывов на три фильма из категории **фильмы ужасов**.

In [94]:
proverka = X_topics[:, 0].argsort()[::-1]
for iter_idx, movie_idx in enumerate(proverka[:3]):
    print(f'\nФильм ужасов #{(iter_idx + 1)}:')
    print(df['review'][movie_idx][:300], '...')


Фильм ужасов #1:
Title: Zombie 3 (1988) <br /><br />Directors: Mostly Lucio Fulci, but also Claudio Fragasso and Bruno Mattei <br /><br />Cast: Ottaviano DellAcqua, Massimo Vani, Beatrice Ring, Deran Serafin <br /><br />Review: <br /><br />To review this flick and get some good background of it, I gotta start by the ...

Фильм ужасов #2:
Over Christmas break, a group of college friends stay behind to help prepare the dorms to be torn down and replaced by apartment buildings. To make the work a bit more difficult, a murderous, Chucks-wearing psycho is wandering the halls of the dorm, preying on the group in various violent ways.<br / ...

Фильм ужасов #3:
Granted, HOTD 2 is better than the Uwe Boll crapfest that was the first one, but thats like saying drowning is better than being chopped alive. OK OK, I'm being a little bit harsh with this one, its just that Video Game adaptations of Zombie movies always leave a bad taste in my mouth. Resident Evil ...


**Фильм ужасов №1:**
**Название:** Zombie 3 (1988)
**Режиссёры:** В основном Лучо Фульчи, а также Клаудио Фрагассо и Бруно Маттеи
**В ролях:** Оттавиано Делл’Аккуа, Массимо Вани, Беатрис Ринг, Деран Серафин
**Обзор:**
Чтобы сделать обзор этого фильма и дать некоторый контекст, мне нужно начать с того, что...

**Фильм ужасов №2:**
На рождественских каникулах группа студентов остаётся, чтобы помочь подготовить общежития к сносу и замене их на жилые дома. Чтобы сделать работу немного сложнее, по коридорам общежития бродит убийца-психопат в обуви Чака, который охотится на группу различными жестокими способами.


**Фильм ужасов №3:**
Конечно, HOTD 2 лучше, чем ужасный фильм Уве Болла, которым был первый, но это всё равно, что сказать «тонуть лучше, чем быть живьём разрубленным». Ладно-ладно, я слишком резок, просто адаптации видеоигр про зомби-фильмы всегда оставляют неприятный осадок во рту. Resident Evil...

---

>Мы посмотрели первые 300 символов для трех наиболее популярных фильмов ужасов. Рецензии - отзывы звучат действительно как для фильмов ужасов.

---


# Итоги:

1. **Анализ настроений (Sentiment Analysis)**

   * Классификация текстов по эмоциональной окраске.
   * Используются алгоритмы машинного обучения, такие как логистическая регрессия и наивный байес.

2. **Представление текста в виде признаков**

   * **Мешок слов (Bag-of-Words):** каждый документ кодируется как вектор частот слов.
   * **TF-IDF:** взвешивание частоты термина по его релевантности, чтобы важные слова имели больший вес.

3. **Особенности работы с текстовыми данными**

   * Большие векторы признаков требуют значительных вычислительных ресурсов.

4. **Моделирование тем (Topic Modeling)**

   * Метод: **Latent Dirichlet Allocation (LDA)**.
   * Применение: распределение обзоров фильмов по темам без меток (обучение без учителя).